# Abstract

This is a reimplementation of the deep-learning method described here: http://forums.fast.ai/t/5th-place-solution-of-favorita-grocery-sales-forecasting/10009

Source is here: https://github.com/LenzDu/Kaggle-Competition-Favorita

# Prelude

## Configuration

In [1]:
DataSetPath = "/home/ec2-user/datasets/favorita/"

StoresPath   = DataSetPath + "stores.csv.gz"
ItemsPath    = DataSetPath + "items.csv.gz"
OilPricePath = DataSetPath + "oil.csv.gz"
HolidaysPath = DataSetPath + "holidays_events.csv.gz"
Transactions = DataSetPath + "transactions.csv.gz"
TrainData    = DataSetPath + "train.csv.gz"
TestData     = DataSetPath + "test.csv.gz"

## Imports

In [2]:
from datetime import date, timedelta

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from datetime import date, timedelta
import gc

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


# Common Functions

This packages up CeShine's code load and reshape the inputs

In [24]:
def load_data(sale_amts_file=TrainData, test_sale_amts_file=TestData, items_file=ItemsPath, stores_file=StoresPath, minYear=2016):
    # df_train = pd.read_feather('train_after1608_raw')
    print ("Loading Train data")
    df_train = pd.read_csv(sale_amts_file, usecols=[1, 2, 3, 4, 5], dtype={'onpromotion': bool},
                           converters={'unit_sales': lambda u: np.log1p(float(u)) if float(u) > 0 else 0},
                           parse_dates=["date"])
    
    for year in range(2013,2018,1):
        print ("adding zero entry for year " + str(year))
        df_train.loc[len(df_train)] = [ \
            pd.Timestamp(str(year) + '-12-25 00:00:00'),
            1,
            1,
            0,
            False
    ]
    
    df_test = pd.read_csv(test_sale_amts_file, usecols=[0, 1, 2, 3, 4], dtype={'onpromotion': bool},
                          parse_dates=["date"]).set_index(['store_nbr', 'item_nbr', 'date'])

    # subset data
    df_2017 = df_train.loc[df_train.date>=pd.datetime(minYear,1,1)]

    # promo
    promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
    promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
    promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
    promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
    promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
    promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
    del promo_2017_test, promo_2017_train

    df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
    df_2017.columns = df_2017.columns.get_level_values(1)

    # items
    items = pd.read_csv(items_file).set_index("item_nbr")
    stores = pd.read_csv(stores_file).set_index("store_nbr")
    # items = items.reindex(df_2017.index.get_level_values(1))

    return df_train, df_2017, promo_2017, items, stores

In [4]:
def save_unstack(df, promo, filename):
    df_name, promo_name = 'df_' + filename + '_raw', 'promo_' + filename + '_raw'
    df.columns = df.columns.astype('str')
    df.reset_index().to_feather(df_name)
    promo.columns = promo.columns.astype('str')
    promo.reset_index().to_feather(promo_name)

def load_unstack(filename):
    df_name, promo_name = 'df_' + filename + '_raw', 'promo_' + filename + '_raw'
    df_2017 = pd.read_feather(df_name).set_index(['store_nbr','item_nbr'])
    df_2017.columns = pd.to_datetime(df_2017.columns)
    promo_2017 = pd.read_feather(promo_name).set_index(['store_nbr','item_nbr'])
    promo_2017.columns = pd.to_datetime(promo_2017.columns)
    items = pd.read_csv(ItemsPath).set_index("item_nbr")
    stores = pd.read_csv(StoresPath).set_index("store_nbr")

    return df_2017, promo_2017, items, stores

In [5]:
# Create validation and test data
def create_dataset(df, promo_df, items, stores, timesteps, first_pred_start, is_train=True, aux_as_tensor=False, reshape_output=0):
    encoder = LabelEncoder()
    items_reindex = items.reindex(df.index.get_level_values(1))
    item_family = encoder.fit_transform(items_reindex['family'].values)
    item_class = encoder.fit_transform(items_reindex['class'].values)
    item_perish = items_reindex['perishable'].values

    stores_reindex = stores.reindex(df.index.get_level_values(0))
    store_nbr = df.reset_index().store_nbr.values - 1
    store_cluster = stores_reindex['cluster'].values - 1
    store_type = encoder.fit_transform(stores_reindex['type'].values)

    # item_mean_df = df.groupby('item_nbr').mean().reindex(df.index.get_level_values(1))
    item_group_mean = df.groupby('item_nbr').mean()
    store_group_mean = df.groupby('store_nbr').mean()
    # store_family_group_mean = df.join(items['family']).groupby(['store_nbr', 'family']).transform('mean')
    # store_family_group_mean.index = df.index

    cat_features = np.stack([item_family, item_class, item_perish, store_nbr, store_cluster, store_type], axis=1)

    return create_dataset_part(df, promo_df, cat_features, item_group_mean, store_group_mean, timesteps, first_pred_start, reshape_output, aux_as_tensor, is_train)


In [6]:
def train_generator(df, promo_df, items, stores, timesteps, first_pred_start,
    n_range=1, day_skip=7, is_train=True, batch_size=2000, aux_as_tensor=False, reshape_output=0, first_pred_start_2016=None):
    encoder = LabelEncoder()
    items_reindex = items.reindex(df.index.get_level_values(1))
    item_family = encoder.fit_transform(items_reindex['family'].values)
    item_class = encoder.fit_transform(items_reindex['class'].values)
    item_perish = items_reindex['perishable'].values

    stores_reindex = stores.reindex(df.index.get_level_values(0))
    store_nbr = df.reset_index().store_nbr.values - 1
    store_cluster = stores_reindex['cluster'].values - 1
    store_type = encoder.fit_transform(stores_reindex['type'].values)

    # item_mean_df = df.groupby('item_nbr').mean().reindex(df.index.get_level_values(1))
    item_group_mean = df.groupby('item_nbr').mean()
    store_group_mean = df.groupby('store_nbr').mean()
    # store_family_group_mean = df.join(items['family']).groupby(['store_nbr', 'family']).transform('mean')
    # store_family_group_mean.index = df.index

    cat_features = np.stack([item_family, item_class, item_perish, store_nbr, store_cluster, store_type], axis=1)

    while 1:
        date_part = np.random.permutation(range(n_range))
        if first_pred_start_2016 is not None:
            range_diff = (first_pred_start - first_pred_start_2016).days / day_skip
            date_part = np.concat([date_part, np.random.permutation(range(range_diff, int(n_range/2) + range_diff))])

        for i in date_part:
            keep_idx = np.random.permutation(df.shape[0])[:batch_size]
            df_tmp = df.iloc[keep_idx,:]
            promo_df_tmp = promo_df.iloc[keep_idx,:]
            cat_features_tmp = cat_features[keep_idx]
            # item_mean_tmp = item_mean_df.iloc[keep_idx, :]

            pred_start = first_pred_start - timedelta(days=int(day_skip*i))

            # Generate a batch of random subset data. All data in the same batch are in the same period.
            yield create_dataset_part(df_tmp, promo_df_tmp, cat_features_tmp, item_group_mean, store_group_mean, timesteps, pred_start, reshape_output, aux_as_tensor, True)

    gc.collect()

In [7]:
def create_dataset_part(df, promo_df, cat_features, item_group_mean, store_group_mean, timesteps, pred_start, reshape_output, aux_as_tensor, is_train, weight=False):

    item_mean_df = item_group_mean.reindex(df.index.get_level_values(1))
    store_mean_df = store_group_mean.reindex(df.index.get_level_values(0))
    # store_family_mean_df = store_family_group_mean.reindex(df.index)

    X, y = create_xy_span(df, pred_start, timesteps, is_train)
    is0 = (X==0).astype('uint8')
    promo = promo_df[pd.date_range(pred_start-timedelta(days=timesteps), periods=timesteps+16)].values
    weekday = np.tile([d.weekday() for d in pd.date_range(pred_start-timedelta(days=timesteps), periods=timesteps+16)],
                          (X.shape[0],1))
    dom = np.tile([d.day-1 for d in pd.date_range(pred_start-timedelta(days=timesteps), periods=timesteps+16)],
                          (X.shape[0],1))
    item_mean, _ = create_xy_span(item_mean_df, pred_start, timesteps, False)
    store_mean, _ = create_xy_span(store_mean_df, pred_start, timesteps, False)
    # store_family_mean, _ = create_xy_span(store_family_mean_df, pred_start, timesteps, False)
    # month_tmp = np.tile([d.month-1 for d in pd.date_range(pred_start-timedelta(days=timesteps), periods=timesteps+16)],
    #                       (X_tmp.shape[0],1))
    yearAgo, _ = create_xy_span(df, pred_start-timedelta(days=365), timesteps+16, False)
    quarterAgo, _ = create_xy_span(df, pred_start-timedelta(days=91), timesteps+16, False)

    if reshape_output>0:
        X = X.reshape(-1, timesteps, 1)
    if reshape_output>1:
        is0 = is0.reshape(-1, timesteps, 1)
        promo = promo.reshape(-1, timesteps+16, 1)
        yearAgo = yearAgo.reshape(-1, timesteps+16, 1)
        quarterAgo = quarterAgo.reshape(-1, timesteps+16, 1)
        item_mean = item_mean.reshape(-1, timesteps, 1)
        store_mean = store_mean.reshape(-1, timesteps, 1)
        # store_family_mean = store_family_mean.reshape(-1, timesteps, 1)

    w = (cat_features[:, 2] * 0.25 + 1) / (cat_features[:, 2] * 0.25 + 1).mean()

    cat_features = np.tile(cat_features[:, None, :], (1, timesteps+16, 1)) if aux_as_tensor else cat_features

    # Use when only 6th-16th days (private periods) are in the training output
    # if is_train: y = y[:, 5:]

    if weight: 
        return ([X, is0, promo, yearAgo, quarterAgo, weekday, dom, cat_features, item_mean, store_mean], y, w)
    else: 
        return ([X, is0, promo, yearAgo, quarterAgo, weekday, dom, cat_features, item_mean, store_mean], y)

In [8]:
def create_xy_span(df, pred_start, timesteps, is_train=True, shift_range=0):
    X = df[pd.date_range(pred_start-timedelta(days=timesteps), pred_start-timedelta(days=1))].values
    if is_train: y = df[pd.date_range(pred_start, periods=16)].values
    else: y = None
    return X, y

# Not used in the final model
def random_shift_slice(mat, start_col, timesteps, shift_range):
    shift = np.random.randint(shift_range+1, size=(mat.shape[0],1))
    shift_window = np.tile(shift,(1,timesteps)) + np.tile(np.arange(start_col, start_col+timesteps),(mat.shape[0],1))
    rows = np.arange(mat.shape[0])
    rows = rows[:,None]
    columns = shift_window
    return mat[rows, columns]

# Calculate RMSE scores for all 16 days, first 5 days (fror public LB) and 6th-16th days (for private LB) 
def cal_score(Ytrue, Yfit):
    print([metrics.mean_squared_error(Ytrue, Yfit), 
    metrics.mean_squared_error(Ytrue[:,:5], Yfit[:,:5]),
    metrics.mean_squared_error(Ytrue[:,5:], Yfit[:,5:])])

# Create submission file
def make_submission(df_index, test_pred, filename):
    df_test = pd.read_csv("test.csv", usecols=[0, 1, 2, 3, 4], dtype={'onpromotion': bool},
                      parse_dates=["date"]).set_index(['store_nbr', 'item_nbr', 'date'])
    df_preds = pd.DataFrame(
        test_pred, index=df_index,
        columns=pd.date_range("2017-08-16", periods=16)
    ).stack().to_frame("unit_sales")
    df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

    submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
    submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
    submission.to_csv(filename, float_format='%.4f', index=None)

# First Pass Over the Data

This just converts the entire dataset to the Feather format for easier access

In [11]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # suppress tf warnings

In [25]:
df, df_unstacked, promo_df, items, stores = load_data(minYear=2013)

Loading Train data


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


adding zero entry for year 2013
adding zero entry for year 2014
adding zero entry for year 2015
adding zero entry for year 2016
adding zero entry for year 2017


2013-01-01  2013-01-02  2013-01-03  2013-01-04  \
store_nbr item_nbr                                                   
1         96995            0.0    0.000000    0.000000    0.000000   
          99197            0.0    0.000000    0.000000    0.000000   
          103520           0.0    0.000000    0.000000    1.098612   
          103665           0.0    1.098612    1.386294    1.098612   
          105574           0.0    2.197225    1.609438    2.197225   
          105575           0.0    2.772589    1.945910    2.197225   
          105577           0.0    1.098612    1.098612    1.386294   
          105693           0.0    0.000000    1.098612    1.098612   
          105737           0.0    1.098612    0.693147    2.079442   
          105857           0.0    2.564949    1.609438    2.397895   
          106716           0.0    1.098612    0.000000    0.000000   
          108079           0.0    0.000000    0.000000    0.000000   
          108634           0.0    0.000000    0.000000    0.000000   
          108696           0.0    1.386294    0.000000    1.098612   
          108698           0.0    1.945910    1.386294    0.693147   
          108701           0.0    1.386294    1.386294    0.693147   
          108786           0.0    0.693147    1.386294    1.386294   
          108797           0.0    0.693147    1.098612    0.693147   
          108831           0.0    3.597641    3.599338    1.681014   
          108862           0.0    0.693147    0.000000    0.000000   
          108952           0.0    1.945910    1.386294    1.098612   
          111223           0.0    2.079442    1.609438    1.098612   
          111397           0.0    2.833213    2.079442    2.995732   
          112830           0.0    1.386294    0.693147    1.386294   
          114778           0.0    2.397895    3.135494    1.791759   
          114790           0.0    1.386294    2.302585    1.609438   
          114799           0.0    0.693147    0.693147    0.693147   
          114800           0.0    2.397895    2.079442    2.484907   
          115267           0.0    2.079442    1.098612    1.098612   
          115611           0.0    3.610918    4.174387    2.995732   
...                        ...         ...         ...         ...   
54        2081064          0.0    0.000000    0.000000    0.000000   
          2081095          0.0    0.000000    0.000000    0.000000   
          2081142          0.0    0.000000    0.000000    0.000000   
          2081161          0.0    0.000000    0.000000    0.000000   
          2083837          0.0    0.000000    0.000000    0.000000   
          2083859          0.0    0.000000    0.000000    0.000000   
          2084278          0.0    0.000000    0.000000    0.000000   
          2084557          0.0    0.000000    0.000000    0.000000   
          2086882          0.0    0.000000    0.000000    0.000000   
          2087374          0.0    0.000000    0.000000    0.000000   
          2087409          0.0    0.000000    0.000000    0.000000   
          2087543          0.0    0.000000    0.000000    0.000000   
          2087567          0.0    0.000000    0.000000    0.000000   
          2087933          0.0    0.000000    0.000000    0.000000   
          2087978          0.0    0.000000    0.000000    0.000000   
          2088922          0.0    0.000000    0.000000    0.000000   
          2089036          0.0    0.000000    0.000000    0.000000   
          2089339          0.0    0.000000    0.000000    0.000000   
          2099230          0.0    0.000000    0.000000    0.000000   
          2101795          0.0    0.000000    0.000000    0.000000   
          2103250          0.0    0.000000    0.000000    0.000000   
          2105347          0.0    0.000000    0.000000    0.000000   
          2106464          0.0    0.000000    0.000000    0.000000   
          2108569          0.0    0.000000    0.000000    0.000000   
          2109909          0.0

In [13]:
#save_unstack(df, promo_df, "all")

In [23]:
df[0]

KeyError: 0

## CNN Approach

In [27]:
pd.date_range?

In [14]:
import numpy as np
import pandas as pd
from datetime import date, timedelta
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras import optimizers

import os

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
df, promo_df, items, stores = load_unstack('all')

In [26]:
df.shape

(125497045, 5)

In [15]:


# data after 2015
df = df[pd.date_range(date(2015,6,1), date(2017,8,15))]
promo_df = promo_df[pd.date_range(date(2015,6,1), date(2017,8,31))]

promo_df = promo_df[df[pd.date_range(date(2017,1,1), date(2017,8,15))].max(axis=1)>0]
df = df[df[pd.date_range(date(2017,1,1), date(2017,8,15))].max(axis=1)>0]
promo_df = promo_df.astype('int')

df_test = pd.read_csv("test.csv", usecols=[0, 1, 2, 3, 4], dtype={'onpromotion': bool},
                      parse_dates=["date"]).set_index(['store_nbr', 'item_nbr', 'date'])
item_nbr_test = df_test.index.get_level_values(1)
item_nbr_train = df.index.get_level_values(1)
item_inter = list(set(item_nbr_train).intersection(set(item_nbr_test)))
df = df.loc[df.index.get_level_values(1).isin(item_inter)]
promo_df = promo_df.loc[promo_df.index.get_level_values(1).isin(item_inter)]

KeyError: "DatetimeIndex(['2015-06-01', '2015-06-02', '2015-06-03', '2015-06-04',\n               '2015-06-05', '2015-06-06', '2015-06-07', '2015-06-08',\n               '2015-06-09', '2015-06-10',\n               ...\n               '2017-08-06', '2017-08-07', '2017-08-08', '2017-08-09',\n               '2017-08-10', '2017-08-11', '2017-08-12', '2017-08-13',\n               '2017-08-14', '2017-08-15'],\n              dtype='datetime64[ns]', length=807, freq='D') not in index"

In [19]:
df['date']value_counts()

SyntaxError: invalid syntax (<ipython-input-19-20b44590cb40>, line 1)